In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [5]:
data = pd.read_csv('../data/Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_geo = LabelEncoder()
data["Gender"] = label_encoder_geo.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder()
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

with open('../models/encoders/scaler_tuning.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open('../models/encoders/label_encoder_geo_tuning.pkl', 'wb') as f:
    pickle.dump(label_encoder_geo, f)

with open('../models/encoders/onehot_encoder_geo_tuning.pkl', 'wb') as f:
    pickle.dump(onehot_encoder_geo, f)

In [7]:
# Define a function to create the model and try different hyperparameters (KerasClassifier)

def create_model(neurons=32, layers=1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons, activation='relu'))
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [15]:
# Create keras classifier
model = KerasClassifier(layers=1, neurons=32, build_fn=create_model, epochs=50, batch_size=10, verbose=1)

In [16]:
# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [17]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Epoch 1/50


c:\Dev\Personal Projects\bank-churn-prediction-ann\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Dev\Personal Projects\bank-churn-prediction-ann\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 754us/step - accuracy: 0.7635 - loss: 0.5211
Epoch 2/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step - accuracy: 0.8132 - loss: 0.4312
Epoch 3/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 776us/step - accuracy: 0.8215 - loss: 0.4121
Epoch 4/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 710us/step - accuracy: 0.8320 - loss: 0.3977
Epoch 5/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 748us/step - accuracy: 0.8387 - loss: 0.3839
Epoch 6/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step - accuracy: 0.8438 - loss: 0.3735
Epoch 7/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 797us/step - accuracy: 0.8455 - loss: 0.3656
Epoch 8/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 814us/step - accuracy: 0.8494 - loss: 0.3599
Epoch 9/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 829us/step - accuracy: 0.8500 - loss: 0.3560
Epoch 10/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 748us/step - accuracy: 0.8549 - loss: 0.3525
Epoch 11/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8549 - loss: 0.3495  
Epoch 12/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 

c:\Dev\Personal Projects\bank-churn-prediction-ann\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Dev\Personal Projects\bank-churn-prediction-ann\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 958us/step - accuracy: 0.7812 - loss: 0.4923
Epoch 2/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8221 - loss: 0.4277  
Epoch 3/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8299 - loss: 0.4150
Epoch 4/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 950us/step - accuracy: 0.8386 - loss: 0.4024
Epoch 5/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8442 - loss: 0.3891
Epoch 6/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8515 - loss: 0.3776
Epoch 7/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step - accuracy: 0.8526 - loss: 0.3689
Epoch 8/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 845us/step - accuracy: 0.8567 - loss: 0.3626
Epoch 9/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step - accuracy: 0.8577 - loss: 0.3578
Epoch 10/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - accuracy: 0.8564 - loss: 0.3555
Epoch 11/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step - accuracy: 0.8588 - loss: 0.3524
Epoch 12/50
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 858

...

534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 714us/step - accuracy: 0.8215 - loss: 0.4207
Epoch 2/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 708us/step - accuracy: 0.8506 - loss: 0.3633
Epoch 3/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - accuracy: 0.8588 - loss: 0.3476
Epoch 4/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 719us/step - accuracy: 0.8571 - loss: 0.3431
Epoch 5/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 828us/step - accuracy: 0.8627 - loss: 0.3345
Epoch 6/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 705us/step - accuracy: 0.8641 - loss: 0.3301
Epoch 7/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - accuracy: 0.8663 - loss: 0.3269
Epoch 8/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 709us/step - accuracy: 0.8667 - loss: 0.3237
Epoch 9/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step - accuracy: 0.8689 - loss: 0.3171
Epoch 10/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step - accuracy: 0.8701 - loss: 0.3154
Epoch 11/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/step - accuracy: 0.8727 - loss: 0.3076
Epoch 12/100
534/534 ━━━━━━━━━━

c:\Dev\Personal Projects\bank-churn-prediction-ann\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Dev\Personal Projects\bank-churn-prediction-ann\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


534/534 ━━━━━━━━━━━━━━━━━━━━ 1s 770us/step - accuracy: 0.8283 - loss: 0.4096
Epoch 2/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - accuracy: 0.8581 - loss: 0.3504
Epoch 3/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 725us/step - accuracy: 0.8620 - loss: 0.3381
Epoch 4/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - accuracy: 0.8658 - loss: 0.3310
Epoch 5/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step - accuracy: 0.8663 - loss: 0.3278
Epoch 6/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step - accuracy: 0.8673 - loss: 0.3229
Epoch 7/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 713us/step - accuracy: 0.8701 - loss: 0.3165
Epoch 8/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 708us/step - accuracy: 0.8691 - loss: 0.3146
Epoch 9/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - accuracy: 0.8718 - loss: 0.3096
Epoch 10/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - accuracy: 0.8757 - loss: 0.3030
Epoch 11/100
534/534 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step - accuracy: 0.8736 - loss: 0.3019
Epoch 12/100
534/534 ━━━━━━━━━━

c:\Dev\Personal Projects\bank-churn-prediction-ann\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Dev\Personal Projects\bank-churn-prediction-ann\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 783us/step - accuracy: 0.7815 - loss: 0.4825
Epoch 2/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 890us/step - accuracy: 0.8126 - loss: 0.4270
Epoch 3/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 845us/step - accuracy: 0.8225 - loss: 0.4093
Epoch 4/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 890us/step - accuracy: 0.8366 - loss: 0.3886
Epoch 5/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 824us/step - accuracy: 0.8486 - loss: 0.3693
Epoch 6/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 775us/step - accuracy: 0.8551 - loss: 0.3579
Epoch 7/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 690us/step - accuracy: 0.8572 - loss: 0.3513
Epoch 8/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8594 - loss: 0.3480
Epoch 9/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 831us/step - accuracy: 0.8577 - loss: 0.3458
Epoch 10/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 719us/step - accuracy: 0.8605 - loss: 0.3436
Epoch 11/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s 729us/step - accuracy: 0.8611 - loss: 0.3428
Epoch 12/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 1s